## Chapter06 機械学習

News Aggregator Data Setを用い、ニュース記事の見出しをビジネス、科学技術、エンターテイメント、健康のカテゴリに分類するタスクに取り組む

In [1]:
!wc -l ./newsCorpora.csv

  422937 ./newsCorpora.csv


In [2]:
!head -3 ./newsCorpora.csv

1	Fed official says weak data caused by weather, should not slow taper	http://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story\?track=rss	Los Angeles Times	b	ddUyU0VZz0BRneMioxUPQVP6sIxvM	www.latimes.com	1394470370698
2	Fed's Charles Plosser sees high bar for change in pace of tapering	http://www.livemint.com/Politics/H2EvwJSK2VE6OF7iK1g3PP/Feds-Charles-Plosser-sees-high-bar-for-change-in-pace-of-ta.html	Livemint	b	ddUyU0VZz0BRneMioxUPQVP6sIxvM	www.livemint.com	1394470371207
3	US open: Stocks fall after Fed official hints at accelerated tapering	http://www.ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436	IFA Magazine	b	ddUyU0VZz0BRneMioxUPQVP6sIxvM	www.ifamagazine.com	1394470371550


### 50. データの入手・整形(2)

News Aggregator Data Setをダウンロードし、以下の要領で学習データ（train.txt），検証データ（valid.txt），評価データ（test.txt）を作成せよ．


1. ダウンロードしたzipファイルを解凍し，readme.txtの説明を読む．
2. 情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
3. 抽出された事例をランダムに並び替える．
4. 抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．


学習データと評価データを作成したら，各カテゴリの事例数を確認せよ

In [3]:
#%%writefile knock50.py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('newsCorpora.csv', sep = '\t', header=None, names = ['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

#要領2 出版元の抽出
flag = df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail'])  #df[].isin(hoge) <- hogeが含まれていたらTrueを返す
df = df[flag]
#まとめて書いた場合が以下の通り
#df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail'])]
#len(df) #13340

#要領3 データのシャッフル
df = df.sample(frac=1, random_state=0)
#sampleメソッド...dfをランダムサンプリング
#frac引数...抽出する行・列の割合。1だと100%指定、再現性確保のためシード値は0に設定

#要領4 データの分割
train_data, other_data = train_test_split(df, test_size=0.2)  # 訓練データ8, その他2
valid_data, test_data = train_test_split(other_data, test_size=0.5)  # 検証データ1, テストデータ1

#テキストに書き出し
#train_data.to_csv('./train.txt', sep = '\t', index = False)
#valid_data.to_csv('./valid.txt', sep = '\t', index = False)
#test_data.to_csv('./test.txt', sep = '\t', index = False)

#各カテゴリの事例数の確認
print('学習データ')
print(train_data['CATEGORY'].value_counts())
print('\n検証データ')
print(valid_data['CATEGORY'].value_counts())
print('\nテストデータ')
print(test_data['CATEGORY'].value_counts())


学習データ
b    4496
e    4213
t    1225
m     738
Name: CATEGORY, dtype: int64

検証データ
b    573
e    539
t    147
m     75
Name: CATEGORY, dtype: int64

テストデータ
b    558
e    527
t    152
m     97
Name: CATEGORY, dtype: int64


###  <span style="color: red; ">51.特徴量抽出(7)</span>

学習データ，検証データ，評価データから特徴量を抽出し，それぞれtrain.feature.txt，valid.feature.txt，test.feature.txtというファイル名で保存せよ． なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．

#### TfidfVectorizer
TfidfVectorizerは文書群を与えると，各文書をtf-Idfの値を元にしたベクトルに変換するもの．

- tf ... term frequency 単語の出現頻度
- idf ... inverse document frequency 


$idf_{wj} = log{\frac{文書数+1}{w_{i}が出現する文書数+1}}+1$


特定の文書に出現する単語ほど、ある話題に特化した意味のある単語という考え方



In [6]:
#%%writefile knock51.py
import pandas as pd
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from knock50 import train_data, valid_data, test_data

def preprosessing(text):
    '''前処理'''
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    text = text.translate(table)
    text = text.lower()
    pattern = re.compile('[0-9]+')
    text = re.sub(pattern, '0', text)

    return text

#データの連結、前処理
df = pd.concat([train_data, valid_data, test_data], axis = 0)
df.reset_index(drop=True, inplace=True)
df['TITLE'] = df['TITLE'].map(lambda x: preprosessing(x)) #map関数を使ってSeriesの各要素に前処理の関数を適用

#単語のベクトル化
vec_tfidf = TfidfVectorizer() #TfidfVectorizerのインスタンス生成
data = vec_tfidf.fit_transform(df['TITLE'])
data = pd.DataFrame(data.toarray(), columns = vec_tfidf.get_feature_names_out())

#分割幅の指定
split_point1 = int(len(data)//3)
split_point2 = int(split_point1 * 2)

#学習、検証、評価データ
x_train = data[:split_point1]
x_valid = data[split_point1:split_point2]
x_test = data[split_point2:]

#学習、検証、評価等別
y_data = df['CATEGORY']
y_train = y_data[:split_point1]
y_valid = y_data[split_point1:split_point2]
y_test = y_data[split_point2:]

#特徴量の書き出し
#x_train.to_csv('train.feature.txt', sep = '\t', index = False)
#x_valid.to_csv('valid.feature.txt', sep = '\t', index = False)
#x_test.to_csv('test.feature.txt', sep = '\t', index = False)

学習データ
b    4495
e    4223
t    1222
m     732
Name: CATEGORY, dtype: int64

検証データ
b    564
e    521
t    157
m     92
Name: CATEGORY, dtype: int64

テストデータ
b    568
e    535
t    145
m     86
Name: CATEGORY, dtype: int64


### 52. 学習 (2)
51で構築した学習データを用いて，ロジスティック回帰モデルを学習せよ．    

In [7]:
#%%writefile knock52.py
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()  # ロジスティック回帰モデルのインスタンス生成
model.fit(x_train, y_train)  # ロジスティック回帰モデルの重みを学習

LogisticRegression()

### 53. 予測(2)
52で学習したロジスティック回帰モデルを用い，与えられた記事見出しからカテゴリとその予測確率を計算するプログラムを実装せよ．

In [8]:
#%%writefile knock53.py
y_pred = model.predict(x_valid)  # 検証データで予測
print('各記事のカテゴリ(ラベル) : ', y_valid.values)
print('各記事のカテゴリ予測 : ', y_pred)

# predict_proba(X)各データがそれぞれのクラスに所属する確率を返す
y_pred = model.predict_proba(x_valid)  
print('カテゴリの予測確率:\n', y_pred)

各記事のカテゴリ(ラベル) :  ['t' 'e' 'e' ... 'e' 'b' 'e']
各記事のカテゴリ予測 :  ['b' 'e' 'e' ... 'e' 'b' 'e']
カテゴリの予測確率:
 [[0.40230371 0.17059931 0.07384669 0.3532503 ]
 [0.15600929 0.72786042 0.06116336 0.05496693]
 [0.1397064  0.66967009 0.08611403 0.10450948]
 ...
 [0.29278451 0.47931844 0.05808103 0.16981602]
 [0.60337228 0.2747074  0.0550837  0.06683662]
 [0.16470895 0.58320611 0.04261545 0.20946949]]


#### 54. 正解率の計算(3)

52で学習したロジスティック回帰モデルの正解率を，学習データおよび評価データ上で計測せよ．

In [9]:
#%%writefile knock54.py
from sklearn.metrics import accuracy_score  # 正解率計算用のメソッド

y_train_pred = model.predict(x_train)  #予測
y_test_pred = model.predict(x_test)

print(f'Accuracy (train) : {accuracy_score(y_train, y_train_pred)}')
print(f'Accuracy (test) : {accuracy_score(y_test, y_test_pred)}')
# Accuracyは正解してたらsum+=1とかしてデータ数で割れば出せる　面倒なのでメソッド使った

Accuracy (train) : 0.9322986954565902
Accuracy (test) : 0.8421762589928058


#### 55. 混同行列の作成(3)

52で学習したロジスティック回帰モデルの混同行列（confusion matrix）を，学習データおよび評価データ上で作成せよ．

In [11]:
#%%writefile knock55.py
from sklearn.metrics import confusion_matrix

# 対角成分の数だけ正解

print(f'Confusion matrix (train)\n: {confusion_matrix(y_train, y_train_pred)}')
print(f'Confusion matrix (test)\n : {confusion_matrix(y_test, y_test_pred)}')


Confusion matrix (train)
: [[1819   24    1    2]
 [   4 1758    0    2]
 [  41   77  208    2]
 [  79   68    1  360]]
Confusion matrix (test)
 : [[1773  129    3   14]
 [  32 1707    1    6]
 [  72  121  103    2]
 [ 155  167    0  163]]


#### 56. 適合率，再現率，F1スコアの計測(5 主に評価指標の概念)
52で学習したロジスティック回帰モデルの適合率，再現率，F1スコアを，評価データ上で計測せよ．カテゴリごとに適合率，再現率，F1スコアを求め，カテゴリごとの性能をマイクロ平均（micro-average）とマクロ平均（macro-average）で統合せよ．

#### TP, FP, FN, TN
- TP(True Positive)  
- FP(False Positive)
- FN(False Negative)
- TN(True Negative)

Trueは予測正解、Falseは予測不正解
Positiveは予測が正、Negativeは予測が負

In [13]:
#%%writefile knock56.py
from sklearn.metrics import precision_score, recall_score, f1_score
# precision_score()  適合率 TP / (TP + FP)
# recall_score() 再現率 TP / (TP + FN)
# f1_score()  F1スコア、適合率と再現率の調和平均

def metrics(y_data, y_pred, ave=None):
  precision_sco = precision_score(y_data, y_pred, average=ave)
  recall_sco = recall_score(y_data, y_pred, average=ave)
  f1_sco = f1_score(y_data, y_pred, average=ave)
  form = "適合率：{}\n再現率 : {}\nF1 : {}\n".format(precision_sco, recall_sco, f1_sco)
  return form

print(f"【カテゴリ順】{model.classes_}\n{metrics(y_test, y_test_pred)}")
print("【マクロ平均】\n", metrics(y_test, y_test_pred, "macro"))
print("【マイクロ平均】\n", metrics(y_test, y_test_pred, "micro"))

【カテゴリ順】['b' 'e' 'm' 't']
適合率：[0.87253937 0.80367232 0.96261682 0.88108108]
再現率 : [0.92391871 0.97766323 0.34563758 0.33608247]
F1 : [0.89749431 0.88217054 0.50864198 0.48656716]

【マクロ平均】
 適合率：0.8799773974934771
再現率 : 0.6458254990050528
F1 : 0.6937184968406463

【マイクロ平均】
 適合率：0.8421762589928058
再現率 : 0.8421762589928058
F1 : 0.8421762589928058



#### 57. 特徴量の重みの確認 (5, 主にargsort, model)
52で学習したロジスティック回帰モデルの中で，重みの高い特徴量トップ10と，重みの低い特徴量トップ10を確認せよ．

- LogisticRegression.classes_ 

    分類したカテゴリのラベル

- LogisticRegression.coef_

    特徴量ごとの重みを取得

- numpy.argsort()

    特定の行(列)をソート、値ではなくインデックスのndarrayを返す

    argsortはデフォルトで昇順の結果でしか受け取ることができない。

        -> 降順で受け取る場合、引数のnumpy配列をマイナス倍



In [57]:
#%%writefile knock57.py
import numpy as np

features = x_train.columns.values  # 学習データの特徴量

class_name, coef = model.classes_[0], model.coef_[0]
index = [i for i in range(1, 11)]

top_10 = features[np.argsort(-coef)[:10]]  # 降順
worst_10 = features[np.argsort(coef)[:10]]

df_top_10 = pd.DataFrame(top_10, columns=[f'重みの高い特徴量トップ10（クラス名: {class_name}）'], index = index)
df_worst_10 = pd.DataFrame(worst_10, columns=[f'重みの低い特徴量トップ10（クラス名: {class_name}）'], index= index)

'''
for class_name, coef in zip(model.classes_, model.coef_):
    index = [i for i in range(1, 11)]
    top_10 = features[np.argsort(-coef)[:10]]
    worst_10 = features[np.argsort(coef)[:10]]
    df_top_10 = pd.DataFrame(top_10, columns=[f'重みの高い特徴量トップ10（クラス名: {class_name}）'], index = index)
    df_worst_10 = pd.DataFrame(worst_10, columns=[f'重みの低い特徴量トップ10（クラス名: {class_name}）'], index= index)
'''

df_top_10.to_csv('./results/output_top10.csv')
df_worst_10.to_csv('./results/output_worst10.csv')

df_top_10     

,重みの高い特徴量トップ10（クラス名: b）
1,us
2,update
3,stocks
4,as
5,euro
6,on
7,china
8,ecb
9,fed
10,dollar


#### 58. 正則化パラメータの変更(3)

ロジスティック回帰モデルを学習するとき，正則化パラメータを調整することで，学習時の過学習（overfitting）の度合いを制御できる．異なる正則化パラメータでロジスティック回帰モデルを学習し，学習データ，検証データ，および評価データ上の正解率を求めよ．実験の結果は，正則化パラメータを横軸，正解率を縦軸としたグラフにまとめよ．

In [49]:
#%%writefile knock58.py
import matplotlib.pyplot as plt
import japanize_matplotlib

# cごとの正解率を格納(縦軸)
accuracies_train = []
accuracies_valid = []
accuracies_test = []

# cは正則化パラメータlambdaの逆数
c_list = [0.001, 0.01, 0.1, 1, 10]  # np.linsapce(start, stop, 要素数)

for c in c_list:
    '''モデルの構築, フィッティング'''
    model = LogisticRegression(C = c, random_state = 0)
    model.fit(x_train, y_train)

    '''予測'''
    # 訓練データ
    y_pred_train = model.predict(x_train) 
    accuracy_train = accuracy_score(y_pred_train, y_train)  
    accuracies_train.append(accuracy_train)

    #検証データ
    y_pred_valid = model.predict(x_valid)
    accuracy_valid = accuracy_score(y_pred_valid, y_valid)  
    accuracies_valid.append(accuracy_valid) 

    #テストデータ
    y_pred_test  = model.predict(x_test)
    accuracy_test = accuracy_score(y_pred_test, y_test)  
    accuracies_test.append(accuracy_test) 

    print(f'正則化パラメータ: {c}')
    print(f'正解率(訓練データ): {accuracy_train}')
    print(f'正解率(検証データ): {accuracy_valid}')
    print(f'正解率(テストデータ): {accuracy_test}')
    print('-'*40)

plt.plot(c_list, accuracies_train, label = 'tarin', marker = 'o')
plt.plot(c_list, accuracies_valid, label = 'valid', marker = 'o')
plt.plot(c_list, accuracies_test, label = 'test', marker = 'o')
plt.xlabel('正則化パラメータc')
plt.ylabel('正解率')
plt.legend()
plt.show()

Writing knock58.py


#### 59. ハイパーパラメータの探索(6)

In [51]:
#%%writefile knock59.py

from sklearn.model_selection import GridSearchCV

params = {'C':[0.01, 0.005, 10]}

gs_model = GridSearchCV(LogisticRegression(
    max_iter=1500),params, cv=5, verbose=1)
gs_model.fit(x_train, y_train)

best_gs_model = gs_model.best_estimator_
print("\ntrain_score: {:.2%}".format(best_gs_model.score(x_train, y_train)))
print("valid_score: {:.2%}".format(best_gs_model.score(x_valid, y_valid)))
print("test_score: {:.2%}".format(best_gs_model.score(x_test, y_test)))

Writing knock59.py
